<a href="https://colab.research.google.com/github/mk6749261-create/Named-Entity-Recognition/blob/main/Another_copy_of_notebook3c4de1b180.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
purusinghvi_email_spam_classification_dataset_path = kagglehub.dataset_download('purusinghvi/email-spam-classification-dataset')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("/kaggle/input/email-spam-classification-dataset/combined_data.csv")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.duplicated().sum()

In [ ]:
df["label"].value_counts()

In [ ]:
df["text"]=df["text"].str.lower()

In [ ]:
import string
df["text"]=df["text"].str.translate(str.maketrans(" "," ",string.punctuation))

In [ ]:
df.head()

In [ ]:
import nltk
nltk.download('punkt_tab')
df["text"]=df["text"].apply(nltk.word_tokenize)

In [ ]:
df.head()

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words=stopwords.words("english")
stop_words.remove("not")

In [ ]:
def remove_stopwords(text):
  return[word for word in text if word not in stop_words]
df["text"] =df["text"].apply(remove_stopwords)

In [ ]:
df.head()

In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
Lemmatizer=WordNetLemmatizer()


In [ ]:
def lemmatize_words(text):
  return[Lemmatizer.lemmatize(word) for word in text]
df["text"]=df["text"].apply(lemmatize_words)

In [ ]:
df.head()

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['text'])


In [ ]:
word_index = tokenizer.word_index
print(word_index)

In [ ]:
sequences=tokenizer.texts_to_sequences(df['text'])
print(sequences[0:5])

In [ ]:
padded=pad_sequences(sequences,maxlen=50,padding='post',truncating='post')
print(padded[0:5])

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(padded,df['label'],test_size=0.2)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=200),
    LSTM(64, return_sequences=False),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(
    x_train, y_train,
    epochs=5,
    batch_size=64,
    validation_split=0.2,
    verbose=1
)

loss, acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {acc:.3f}")


In [ ]:
from sklearn.metrics import classification_report
y_pred=model.predict(x_test)
y_pred_classes=(y_pred>0.5).astype("int32")
print(classification_report(y_pred_classes,y_test))

In [ ]:
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
cm=confusion_matrix(y_test,y_pred_classes)
print (cm)

In [ ]:
import pandas as pd

counts = pd.Series(y_pred_classes.flatten()).value_counts()
percentages = counts / len(y_pred_classes) * 100

print("num-of-words")
print(counts)

print("persentage")
print(percentages)
